# Individual Assignment 3

Name: Mastroianni, Michael, mmastroianni@ryerson.ca

This assignment extends the in-class exercise on multiple linear regression analysis. You are allowed to use all resources you can find, including lecture slides, in-class examples, previous group assignments, LinkedIn training courses, and the internet.

You are **not allowed** to discuss this assignment with anyone other than the course instructor. This assignment is required to be your individual work. 

Completed assignment should be merged to the main branch in your repository.

***
## Data Description

The data set is contained in a comma-separated value (csv) file named ```CDN_hprice.csv``` with column headers. 

The data is a set of quarterly observations on a housing price index and other relevant variables in Canada for 1976 through 2019.

This data set is obtained from [Federal Reserve Bank of Dallas's International House Price Database](https://www.dallasfed.org/institute/houseprice).

Description of the data is as follow:

| Variable name | Variable description |
| -- | ----------- |
| Year      | Year |
| Quarter   | Quarter |
| RHPI      | House Price Index (real) |
| RPDI      | Personal disposable income (real) |
| logRHPI   | log(RHPI)   |
| logRPDI   | log(RPDI)   |
| UE        | Unemployment rate (in percentage points) |
| CPI       | Consumer Price Index |
| TB10_rt   | 10-year treasury bond rate (nominal) |
| RTB10_rt  | 10-year Treasury Bond rate (real) |


***
## Load the required modules

In [17]:
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot

***
## Import the data set

#### Load the data set into Python

In [2]:
hprice = pd.read_csv("CDN_hprice.csv")
hprice.head()

,Year,Quarter,RHPI,RPDI,RTB10_rt,CPI,TB10_rt,UE,logRHPI,logRPDI
0,1976,Q1,58.98,74.11,0.835,24.414,9.327,6.867,4.0772,4.3056
1,1976,Q2,59.81,75.93,2.535,24.783,9.270,7.167,4.0912,4.3298
2,1976,Q3,58.82,73.23,2.885,25.125,8.937,7.367,4.0745,4.2936
3,1976,Q4,57.63,75.18,1.919,25.652,8.610,7.733,4.0540,4.3199
4,1977,Q1,55.78,74.07,0.943,26.310,8.783,7.867,4.0214,4.3050


***
## Question 1

#### 1.1 Sort the data in ascending order by year and quarter

In [3]:
hprice = hprice.sort_values(by=["Year","Quarter"])
print(hprice.head(10))

   Year Quarter   RHPI   RPDI  RTB10_rt     CPI  TB10_rt     UE  logRHPI  \
0  1976      Q1  58.98  74.11     0.835  24.414    9.327  6.867   4.0772   
1  1976      Q2  59.81  75.93     2.535  24.783    9.270  7.167   4.0912   
2  1976      Q3  58.82  73.23     2.885  25.125    8.937  7.367   4.0745   
3  1976      Q4  57.63  75.18     1.919  25.652    8.610  7.733   4.0540   
4  1977      Q1  55.78  74.07     0.943  26.310    8.783  7.867   4.0214   
5  1977      Q2  55.20  75.11     0.238  26.863    8.653  8.233   4.0110   
6  1977      Q3  54.45  73.77    -0.360  27.416    8.727  8.400   3.9973   
7  1977      Q4  56.11  74.34     0.044  27.969    9.083  8.367   4.0273   
8  1978      Q1  55.73  76.48     0.366  28.628    9.207  8.467   4.0205   
9  1978      Q2  56.04  75.70    -0.051  29.339    9.160  8.467   4.0261   

   logRPDI  
0   4.3056  
1   4.3298  
2   4.2936  
3   4.3199  
4   4.3050  
5   4.3190  
6   4.3010  
7   4.3086  
8   4.3370  
9   4.3268  


#### 1.2 Generate time index

Create new variable ```t``` such that $t=0$ in the first period. 

Note that we have repeated year and quarters. You can create the the required index by:
1. Sorting the observations in ascending order, as you are asked to do in (1.1)
2. Add a new column using the ```df.index``` method, where ```df``` is the name of the pandas dataframe

In [4]:
hprice["t"] = hprice.index
print(hprice.head())

   Year Quarter   RHPI   RPDI  RTB10_rt     CPI  TB10_rt     UE  logRHPI  \
0  1976      Q1  58.98  74.11     0.835  24.414    9.327  6.867   4.0772   
1  1976      Q2  59.81  75.93     2.535  24.783    9.270  7.167   4.0912   
2  1976      Q3  58.82  73.23     2.885  25.125    8.937  7.367   4.0745   
3  1976      Q4  57.63  75.18     1.919  25.652    8.610  7.733   4.0540   
4  1977      Q1  55.78  74.07     0.943  26.310    8.783  7.867   4.0214   

   logRPDI  t  
0   4.3056  0  
1   4.3298  1  
2   4.2936  2  
3   4.3199  3  
4   4.3050  4  


#### 1.3 Generate lag variable

Here we create lag variables with a lag of **4** period. That is, in quarterly frequency, the lag variable is in the *same* quarter but *different* year.

Create a new column in the date set named ```logRHPI_4```, such that $\text{logRHPI}\_4 = \log(\textit{RHPI}_{t-4})$ 

In [6]:
hprice["logRHPI_4"] = hprice["logRHPI"].shift(4)
print(hprice.head(8))

   Year Quarter   RHPI   RPDI  RTB10_rt     CPI  TB10_rt     UE  logRHPI  \
0  1976      Q1  58.98  74.11     0.835  24.414    9.327  6.867   4.0772   
1  1976      Q2  59.81  75.93     2.535  24.783    9.270  7.167   4.0912   
2  1976      Q3  58.82  73.23     2.885  25.125    8.937  7.367   4.0745   
3  1976      Q4  57.63  75.18     1.919  25.652    8.610  7.733   4.0540   
4  1977      Q1  55.78  74.07     0.943  26.310    8.783  7.867   4.0214   
5  1977      Q2  55.20  75.11     0.238  26.863    8.653  8.233   4.0110   
6  1977      Q3  54.45  73.77    -0.360  27.416    8.727  8.400   3.9973   
7  1977      Q4  56.11  74.34     0.044  27.969    9.083  8.367   4.0273   

   logRPDI  t  logRHPI_4  
0   4.3056  0        NaN  
1   4.3298  1        NaN  
2   4.2936  2        NaN  
3   4.3199  3        NaN  
4   4.3050  4     4.0772  
5   4.3190  5     4.0912  
6   4.3010  6     4.0745  
7   4.3086  7     4.0540  


Create a new column in the date set named ```logRPDI_4```, such that $\text{logRPDI}\_4 = \log(\textit{RPDI}_{t-4})$ 

In [7]:
hprice["logRPDI_4"] = hprice["logRPDI"].shift(4)
print(hprice.head(6))

   Year Quarter   RHPI   RPDI  RTB10_rt     CPI  TB10_rt     UE  logRHPI  \
0  1976      Q1  58.98  74.11     0.835  24.414    9.327  6.867   4.0772   
1  1976      Q2  59.81  75.93     2.535  24.783    9.270  7.167   4.0912   
2  1976      Q3  58.82  73.23     2.885  25.125    8.937  7.367   4.0745   
3  1976      Q4  57.63  75.18     1.919  25.652    8.610  7.733   4.0540   
4  1977      Q1  55.78  74.07     0.943  26.310    8.783  7.867   4.0214   
5  1977      Q2  55.20  75.11     0.238  26.863    8.653  8.233   4.0110   

   logRPDI  t  logRHPI_4  logRPDI_4  
0   4.3056  0        NaN        NaN  
1   4.3298  1        NaN        NaN  
2   4.2936  2        NaN        NaN  
3   4.3199  3        NaN        NaN  
4   4.3050  4     4.0772     4.3056  
5   4.3190  5     4.0912     4.3298  


#### 1.4 Generate "*first-differencing*" variables

It is also known as the year-over-year difference, because we take the difference between now and 4 quarters (a year) ago. 

E.g. we are calulating the change from 2018 Q1 to 2019 Q1. 

Create a new column named ```gRHPI```, such that $\textit{gRHPI}_t = \Delta \log(\textit{RHPI}_t) = \log(\textit{RHPI}_t) - \log(\textit{RHPI}_{t-4})$

In [11]:
hprice["gRHPI"] = hprice["logRHPI"] - hprice["logRHPI_4"]
print(hprice.head(8))

   Year Quarter   RHPI   RPDI  RTB10_rt     CPI  TB10_rt     UE  logRHPI  \
0  1976      Q1  58.98  74.11     0.835  24.414    9.327  6.867   4.0772   
1  1976      Q2  59.81  75.93     2.535  24.783    9.270  7.167   4.0912   
2  1976      Q3  58.82  73.23     2.885  25.125    8.937  7.367   4.0745   
3  1976      Q4  57.63  75.18     1.919  25.652    8.610  7.733   4.0540   
4  1977      Q1  55.78  74.07     0.943  26.310    8.783  7.867   4.0214   
5  1977      Q2  55.20  75.11     0.238  26.863    8.653  8.233   4.0110   
6  1977      Q3  54.45  73.77    -0.360  27.416    8.727  8.400   3.9973   
7  1977      Q4  56.11  74.34     0.044  27.969    9.083  8.367   4.0273   

   logRPDI  t  logRHPI_4  logRPDI_4   gPHPI   gRHPI  
0   4.3056  0        NaN        NaN     NaN     NaN  
1   4.3298  1        NaN        NaN     NaN     NaN  
2   4.2936  2        NaN        NaN     NaN     NaN  
3   4.3199  3        NaN        NaN     NaN     NaN  
4   4.3050  4     4.0772     4.3056 -0.0558 -

***
## Question 2

Consider the following model for house price growth:
$$\textit{gRHPI}_t = \beta_0 + \beta_1 t + \delta_1 Q2 + \delta_2 Q3 + \delta_3 Q4 + u_t$$

- $t = 0, 1, 2, \ldots$ is the time index you created in (1.2)
- Q2, Q3, Q4 are dummy variables that equals to 1 in the second, third, and fourth quarter, respectively. Otherwise, they are equal to 0. 

#### 2.1 Estimate the model

In [18]:
model_spec = smf.ols(formula = "gRHPI ~ 1 + t + C(Quarter)", data = hprice)
model_est = model_spec.fit()

#### 2.2 Get the estimation results

In [19]:
print(model_est.summary())

                            OLS Regression Results                            
Dep. Variable:                  gRHPI   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.370
Date:                Mon, 07 Dec 2020   Prob (F-statistic):              0.247
Time:                        20:07:58   Log-Likelihood:                 211.10
No. Observations:                 172   AIC:                            -412.2
Df Residuals:                     167   BIC:                            -396.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0033      0.015  

#### 2.3 What is the benchmark (base period) in this model?

the benchmark period or base period references when all variables are equal to 0. In this model the base period is in quarter 1 when t is equal to 0. quarter 1 would be 1976. 

#### 2.4 At 5% significance level, is there a statistically significant time trend?

Yes. there is a time trend because the p-va;ue is smaller than 0.05.

#### 2.5 At 5% significance level, would you conclude that house price growth exhibits seasonality at quarterly frequency?

I would not. This is because the p-values for the quarter variables are greater than 0.05.

***
## Question 3

Consider a house price model with AR(1) specification:
$$\textit{gRHPI}_t = \beta_0 + \beta_1 t + \beta_2 \textit{gRHPI}_{t-1} + \delta_1 Q2 + \delta_2 Q3 + \delta_3 Q4 + u_t$$

#### 3.1 Create a new column in the date set named ```gRHPI_1```, such that $\textit{gRHPI}\_1 = \textit{gRHPI}_{t-1}$ 

In [20]:
hprice['gRHPI_1'] = hprice["gRHPI"].shift(1)
print(hprice.head(6))

   Year Quarter   RHPI   RPDI  RTB10_rt     CPI  TB10_rt     UE  logRHPI  \
0  1976      Q1  58.98  74.11     0.835  24.414    9.327  6.867   4.0772   
1  1976      Q2  59.81  75.93     2.535  24.783    9.270  7.167   4.0912   
2  1976      Q3  58.82  73.23     2.885  25.125    8.937  7.367   4.0745   
3  1976      Q4  57.63  75.18     1.919  25.652    8.610  7.733   4.0540   
4  1977      Q1  55.78  74.07     0.943  26.310    8.783  7.867   4.0214   
5  1977      Q2  55.20  75.11     0.238  26.863    8.653  8.233   4.0110   

   logRPDI  t  logRHPI_4  logRPDI_4   gPHPI   gRHPI  gRHPI_1  
0   4.3056  0        NaN        NaN     NaN     NaN      NaN  
1   4.3298  1        NaN        NaN     NaN     NaN      NaN  
2   4.2936  2        NaN        NaN     NaN     NaN      NaN  
3   4.3199  3        NaN        NaN     NaN     NaN      NaN  
4   4.3050  4     4.0772     4.3056 -0.0558 -0.0558      NaN  
5   4.3190  5     4.0912     4.3298 -0.0802 -0.0802  -0.0558  


#### 3.2 Estimate the model

In [21]:
model_spec = smf.ols(formula = "gRHPI ~ 1 + t + C(Quarter) + gRHPI_1", data = hprice)
model_est = model_spec.fit()

#### 3.3 Get the estimation results

In [22]:
print(model_est.summary())

                            OLS Regression Results                            
Dep. Variable:                  gRHPI   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     147.1
Date:                Mon, 07 Dec 2020   Prob (F-statistic):           6.78e-59
Time:                        20:15:02   Log-Likelihood:                 352.36
No. Observations:                 171   AIC:                            -692.7
Df Residuals:                     165   BIC:                            -673.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0028      0.007  

#### 3.4 At 5% significance level, what would you conclude about $\beta_2$?

i believe we would reject. this is because house prices from a year ago are unknown. 

#### 3.5 Does the AR(1) process have weak dependence?

yes because beta is equal to 0

#### 3.6 Is the time trend still significant? Why or why not?

yes it is still significant at 5%. the estimate is indeed smaller though. 

***
## Question 4

Consider a house price model with AR(1) specification and other exogenous regressors:
$$\textit{gRHPI}_t = \beta_0 + \beta_1 t + \beta_2 \textit{gRHPI}_{t-1} + \beta_3 \text{UE}_t + \delta_1 Q2 + \delta_2 Q3 + \delta_3 Q4 + u_t$$

#### 4.1 Estimate the model

In [24]:
model_spec = smf.ols(formula = "gRHPI ~ 1 + t + C(Quarter) + gRHPI_1 + UE", data = hprice)
model_est = model_spec.fit()

#### 4.2 Get the estimation results

In [25]:
print(model_est.summary())

                            OLS Regression Results                            
Dep. Variable:                  gRHPI   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     133.7
Date:                Mon, 07 Dec 2020   Prob (F-statistic):           1.68e-60
Time:                        20:21:02   Log-Likelihood:                 358.90
No. Observations:                 171   AIC:                            -703.8
Df Residuals:                     164   BIC:                            -681.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0799      0.022  

#### 4.3 How would you interpret $\beta_3$? (Be careful on how unemployment rate is reported in the data set)

when a unit of unemployment increases than the unit gRHPI would be affected by -0.759. This means that as unemployment raises one house prices decrase by 0.759%.

#### 4.4 At 5% significance level, what would you conclude about $\beta_3$?

we would reject the hypothosis that employment has no effect on house prices.

#### 4.5 Explain why the AR(1) dependence ($\beta_2$) is weakened, when compare with your results in Question 3.

This could be because of the added variable in question 4. That would mean that question 3 AR(1) would have less variables to depend on therefore it is weighted heavier. 

***
## References
- Jeffrey M. Wooldridge (2012). "Introductory Econometrics: A Modern Approach, 5e" Chapter 11.
    
- Seabold, Skipper, and Josef Perktold (2010). "[statsmodels: Econometric and statistical modeling with python](https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html)." Proceedings of the 9th Python in Science Conference.